In [1]:
import scipy.io
import numpy as np
mnist = scipy.io.loadmat("C:\\Users\\User\\Desktop\\Internship\\mnist-original.mat")
print(mnist["data"].shape)
print(mnist["target"].shape)

(784, 70000)
(1, 70000)


In [2]:
from sklearn.model_selection import train_test_split


In [3]:
print(np.ravel(mnist["target"]).shape)

(70000,)


In [4]:
# X_train + validation training set (60000, 784)
# X_test test set (10000, 784)
# y_train_val labels of the train set (60000,) (vector)
# y_test labels of the test set 
#First, divide the data set into (train+validation) and test. Then devide (train+val) into train and val 
#train 50000, test 100000, val 100000
X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist["data"].T, np.ravel(mnist["target"]), test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=42)

#print(X_train_val)
#print(X_test)
#print("Labels")
#print(y_train_val)
#print(y_test)

In [5]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

In [6]:
#apriori probabilities for each number 
apriori_array = np.zeros([10,1])
for i in range(0,10):
    apriori_array[i] = np.prod(mnist["target"][mnist["target"]==i].shape)
    
apriori_array = apriori_array / np.prod(mnist["target"].shape)
#print(apriori_array)

In [7]:
random_forest_clf = RandomForestClassifier(random_state=42)
extra_trees_clf = ExtraTreesClassifier(random_state=42)
svm_clf = svm.SVC(random_state=42,probability = True)
mlp_clf = MLPClassifier(random_state=42)
gaussianNB_clf = GaussianNB(apriori_array)

In [ ]:
estimators = [random_forest_clf, extra_trees_clf,svm_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
Training the ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)
Training the SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probabilit

In [ ]:
##score() method measures the quality of the predictions given a test set
# где модели делают предикшены? когда вызывается .score()
value = []
print("Quality of the predictions:")
[estimator.score(X_val, y_val) for estimator in estimators]

for estimator in estimators:
    print(estimator.score(X_val, y_val))
    
print("Weights:")
for estimator in estimators:
    exp_weight=np.exp(10*estimator.score(X_val, y_val))
    value.append(np.exp(10*estimator.score(X_val, y_val)))
print(value)
sum_value=sum(value)
#print("Normalized weights:")
#for estimator in estimators:
    #exp_weight=np.exp(10*estimator.score(X_val, y_val))/sum_value
    #print(exp_weight)

#print missclassified elements     

for estimator in estimators:
    print(y_val[estimator.predict(X_val) != y_val])
    print('____________________________')

# Stacking of first level learners

In [ ]:
normalized_weights = (value/sum_value)
print(normalized_weights)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
#predict_proba(X_val) returns probability estimates for the test vector X, e.g. predcit elements 0-9 and provide its probability
#predict computes argmax of  predic_prob
log_clf = LogisticRegression(random_state=42)
i = 0
for estimator in estimators:
    prob_for_est = estimator.predict_proba(X_val) 
    prob_for_est *= normalized_weights[i]
    print(prob_for_est.shape)
    if i == 0:
        X_train2 = prob_for_est
    else:
        X_train2 = np.hstack((X_train2, prob_for_est))
    i+=1
    
#print(X_train2.shape)
    
y_train2 = y_val
    
log_clf.fit(X_train2, y_train2)

score = cross_val_score(log_clf, X_train2, y_train2, cv=3, verbose=3)
mean_score = score.mean()
print("Blender mean score on the training dataset = %f" % mean_score)


In [ ]:
i=0
for estimator in estimators:
    prob_for_est = estimator.predict_proba(X_test) 
    prob_for_est *= normalized_weights[i]
    #print(prob_for_est.shape)
    if i == 0:
        X_test2 = prob_for_est
    else:
        X_test2 = np.hstack((X_test2, prob_for_est))
    i+=1
    
#print(X_train2.shape)
    
y_test2 = y_test
    
#log_clf.predict(X_test2, y_test2)

score = cross_val_score(log_clf, X_test2, y_test2, cv=3, verbose=3)
mean_score = score.mean()
print("Blender mean score for a testing dataset set = %f" % mean_score)


# Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=4,
    max_samples=100, bootstrap=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)
#y_pred = bag_clf.predict(X_test)
#score = cross_val_score(bag_clf, X_train2, y_train2, cv=3, verbose=3)
#mean_score = score.mean()
#print("Mean score for training of the 2nd level learner = %f" % mean_score)
score = cross_val_score(bag_clf, X_test, y_test, cv=3, verbose=3)
mean_score = score.mean()
print("Mean score for test set of the 2nd level learner = %f" % mean_score)


# Boosting AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=4,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)
score = cross_val_score(ada_clf, X_test, y_test, cv=3, verbose=3)
mean_score = score.mean()
print("Mean score for test set of the 2nd level learner = %f" % mean_score)

# Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("gaussianNB_clf", gaussianNB_clf),
    ("mlp_clf", mlp_clf),
]
voting_clf = VotingClassifier(named_estimators)

In [ ]:
voting_clf.fit(X_train, y_train)

In [ ]:
voting_clf.score(X_test, y_test)

# Voting Classifier SOFT

In [ ]:
voting_clf.voting = "soft"

In [ ]:
voting_clf.score(X_val, y_val)

In [ ]:
voting_clf.score(X_test, y_test)